In [23]:
pip install transformers

In [24]:
!pip install transformers rouge-score nltk bert-score

In [25]:
import json
from transformers import pipeline
from google.colab import files
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score

In [26]:
# Upload the file
uploaded = files.upload()

Saving NuclearDataset.json to NuclearDataset (2).json


In [27]:
# Assume the uploaded file is named 'input.json'
input_file = list(uploaded.keys())[0]

In [31]:
# Load the dataset from the uploaded JSON file
with open(input_file, "r") as f:
    data = json.load(f)

In [33]:
# Load the pre-trained models for question generation and answering
question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [34]:
# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [35]:
output_data = []

In [36]:
# Generate questions and answers
for entry in data["NuclearDataset"]:
    text = entry["Text"]
    context = entry["Context"]

    # Generate two distinct questions
    questions = question_generator(f"generate question: {context}", num_return_sequences=2, num_beams=4, max_length=50)
    generated_questions = [q['generated_text'] for q in questions]

    for generated_question in generated_questions:
        # Ensure the generated question is complete
        if not generated_question.endswith('?'):
            generated_question += "?"

        # Generate answer
        generated_answer = qa_pipeline(question=generated_question, context=context)['answer']

        # Evaluate BLEU score for the generated question
        bleu_score = sentence_bleu([context.split()], generated_question.split())

        # Evaluate ROUGE score for the generated question
        rouge_score = scorer.score(context, generated_question)

        # Evaluate BERTScore
        P, R, F1 = bert_score.score([generated_answer], [context], lang='en')

        output_data.append({
            "Text": text,
            "Context": context,
            "Question": generated_question,
            "Answer": generated_answer,
            "BLEU_Score": bleu_score,
            "ROUGE_Score": rouge_score['rougeL'].fmeasure,
            "BERTScore": F1.mean().item()
        })

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [37]:
# Save to a JSON file
output_file = "output.json"
with open(output_file, "w") as f:
    json.dump(output_data, f, indent=4)

In [38]:
print(f"Output saved to {output_file}")

# Download the output file
files.download(output_file)

Output saved to output.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>